# 📊 Exploratory Data Analysis (EDA)

## Social Media ROI Attribution & Influencer Performance Analyzer

This notebook performs comprehensive EDA to answer key business questions:
- What's our current social media ROI?
- Which platforms drive the most engagement and revenue?
- When do posts perform best?
- Which content types get the most saves (purchase intent)?
- How do influencer tiers compare in effectiveness?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Style settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✅ Libraries loaded!")

In [ ]:
# Load datasets
data_dir = Path("../data/raw")

brands = pd.read_csv(data_dir / "brands.csv")
influencers = pd.read_csv(data_dir / "influencers.csv")
posts = pd.read_csv(data_dir / "posts.csv")
conversions = pd.read_csv(data_dir / "conversions.csv")
touchpoints = pd.read_csv(data_dir / "touchpoints.csv")

# Parse dates
posts['post_date'] = pd.to_datetime(posts['post_date'])
conversions['conversion_date'] = pd.to_datetime(conversions['conversion_date'])
touchpoints['touchpoint_date'] = pd.to_datetime(touchpoints['touchpoint_date'])

print("📊 Datasets loaded:")
print(f"   Brands: {len(brands):,}")
print(f"   Influencers: {len(influencers):,}")
print(f"   Posts: {len(posts):,}")
print(f"   Conversions: {len(conversions):,}")
print(f"   Touchpoints: {len(touchpoints):,}")

---
## 1. Executive Summary Metrics

In [ ]:
# Calculate key metrics
total_revenue = conversions['order_value'].sum()
total_spend = brands['monthly_social_budget'].sum() * 12  # Annual
roi = (total_revenue - total_spend) / total_spend * 100

avg_engagement = posts['likes'].sum() + posts['comments'].sum() + posts['shares'].sum() + posts['saves'].sum()
avg_engagement_rate = (avg_engagement / posts['reach'].sum()) * 100

influencer_revenue = conversions[conversions['influencer_id'].notna()]['order_value'].sum()
influencer_attribution_rate = influencer_revenue / total_revenue * 100

print("📈 EXECUTIVE SUMMARY")
print("=" * 50)
print(f"\n💰 Total Revenue (Attributed): ${total_revenue:,.2f}")
print(f"💸 Total Social Media Spend: ${total_spend:,.2f}")
print(f"📊 Overall ROI: {roi:.1f}%")
print(f"\n👥 Total Influencers: {len(influencers):,}")
print(f"📱 Total Posts: {len(posts):,}")
print(f"🛒 Total Conversions: {len(conversions):,}")
print(f"\n🎯 Influencer Attribution Rate: {influencer_attribution_rate:.1f}%")
print(f"💫 Average Engagement Rate: {avg_engagement_rate:.2f}%")

---
## 2. Platform Performance Analysis

In [ ]:
# Platform metrics
platform_metrics = posts.groupby('platform').agg({
    'post_id': 'count',
    'likes': 'sum',
    'comments': 'sum',
    'shares': 'sum',
    'saves': 'sum',
    'reach': 'sum',
    'impressions': 'sum'
}).rename(columns={'post_id': 'posts'})

platform_metrics['total_engagement'] = platform_metrics['likes'] + platform_metrics['comments'] + platform_metrics['shares'] + platform_metrics['saves']
platform_metrics['engagement_rate'] = (platform_metrics['total_engagement'] / platform_metrics['reach'] * 100).round(2)
platform_metrics['save_rate'] = (platform_metrics['saves'] / platform_metrics['likes'] * 100).round(2)

# Conversions by platform
platform_revenue = posts.merge(conversions[['post_id', 'order_value']], on='post_id', how='inner')
platform_revenue = platform_revenue.groupby('platform')['order_value'].sum()
platform_metrics['revenue'] = platform_revenue

print("📱 Platform Performance")
print(platform_metrics[['posts', 'total_engagement', 'engagement_rate', 'save_rate', 'revenue']].round(2))

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Posts by Platform
colors = sns.color_palette('husl', 4)
platform_metrics['posts'].plot(kind='bar', ax=axes[0], color=colors)
axes[0].set_title('Posts by Platform', fontweight='bold', fontsize=14)
axes[0].set_ylabel('Number of Posts')
axes[0].tick_params(axis='x', rotation=45)

# Engagement Rate by Platform
platform_metrics['engagement_rate'].plot(kind='bar', ax=axes[1], color=colors)
axes[1].set_title('Engagement Rate by Platform', fontweight='bold', fontsize=14)
axes[1].set_ylabel('Engagement Rate (%)')
axes[1].tick_params(axis='x', rotation=45)

# Save Rate (Purchase Intent) by Platform
platform_metrics['save_rate'].plot(kind='bar', ax=axes[2], color=colors)
axes[2].set_title('Save Rate (Purchase Intent) by Platform', fontweight='bold', fontsize=14)
axes[2].set_ylabel('Save Rate (%)')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../data/platform_performance.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 3. Influencer Tier Analysis

In [ ]:
# Merge influencer data with posts
posts_with_inf = posts.merge(influencers[['influencer_id', 'tier', 'follower_count', 'avg_collaboration_cost']], on='influencer_id')

# Tier metrics
tier_order = ['nano', 'micro', 'mid', 'macro', 'mega']
tier_metrics = posts_with_inf.groupby('tier').agg({
    'post_id': 'count',
    'likes': 'mean',
    'comments': 'mean',
    'saves': 'mean',
    'reach': 'mean',
    'avg_collaboration_cost': 'mean'
}).reindex(tier_order)

tier_metrics['engagement_rate'] = ((tier_metrics['likes'] + tier_metrics['comments'] + tier_metrics['saves']) / tier_metrics['reach'] * 100).round(2)

# Revenue by tier
tier_revenue = posts_with_inf.merge(conversions[['post_id', 'order_value']], on='post_id', how='inner')
tier_revenue = tier_revenue.groupby('tier').agg({
    'order_value': ['sum', 'count']
}).reindex(tier_order)
tier_revenue.columns = ['total_revenue', 'conversions']

tier_metrics = tier_metrics.join(tier_revenue)
tier_metrics['cost_per_conversion'] = (tier_metrics['avg_collaboration_cost'] / (tier_metrics['conversions'] / tier_metrics['post_id'])).round(2)
tier_metrics['roi'] = ((tier_metrics['total_revenue'] - tier_metrics['avg_collaboration_cost'] * tier_metrics['post_id']) / (tier_metrics['avg_collaboration_cost'] * tier_metrics['post_id']) * 100).round(1)

print("👤 Influencer Tier Performance")
print(tier_metrics[['post_id', 'engagement_rate', 'avg_collaboration_cost', 'total_revenue', 'roi']].round(2))

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Engagement Rate by Tier
colors = sns.color_palette('RdYlGn', 5)
tier_metrics['engagement_rate'].plot(kind='bar', ax=axes[0, 0], color=colors)
axes[0, 0].set_title('Engagement Rate by Influencer Tier', fontweight='bold', fontsize=14)
axes[0, 0].set_ylabel('Engagement Rate (%)')
axes[0, 0].tick_params(axis='x', rotation=45)

# Cost vs ROI
axes[0, 1].bar(tier_metrics.index, tier_metrics['roi'], color=colors)
axes[0, 1].set_title('ROI by Influencer Tier', fontweight='bold', fontsize=14)
axes[0, 1].set_ylabel('ROI (%)')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].axhline(y=0, color='red', linestyle='--', alpha=0.5)

# Average Cost by Tier
tier_metrics['avg_collaboration_cost'].plot(kind='bar', ax=axes[1, 0], color=colors)
axes[1, 0].set_title('Average Cost per Post by Tier', fontweight='bold', fontsize=14)
axes[1, 0].set_ylabel('Cost (USD)')
axes[1, 0].tick_params(axis='x', rotation=45)

# Revenue by Tier
tier_metrics['total_revenue'].plot(kind='bar', ax=axes[1, 1], color=colors)
axes[1, 1].set_title('Total Revenue by Tier', fontweight='bold', fontsize=14)
axes[1, 1].set_ylabel('Revenue (USD)')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../data/tier_performance.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 4. Optimal Posting Time Analysis

In [ ]:
# Calculate engagement by hour and day
posts['total_engagement'] = posts['likes'] + posts['comments'] + posts['shares'] + posts['saves']
posts['engagement_rate'] = posts['total_engagement'] / posts['reach'] * 100

# Day names
day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

# Heatmap data
heatmap_data = posts.groupby(['day_of_week', 'post_time_hour'])['engagement_rate'].mean().unstack()

# Rename index to day names
heatmap_data.index = [day_names[i] for i in heatmap_data.index]

fig, ax = plt.subplots(figsize=(16, 6))
sns.heatmap(heatmap_data, cmap='RdYlGn', annot=False, fmt='.1f', ax=ax, cbar_kws={'label': 'Engagement Rate (%)'})
ax.set_title('Optimal Posting Times: Engagement Rate Heatmap', fontweight='bold', fontsize=16)
ax.set_xlabel('Hour of Day', fontsize=12)
ax.set_ylabel('Day of Week', fontsize=12)

plt.tight_layout()
plt.savefig('../data/posting_times_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

# Best times
best_times = posts.groupby(['day_of_week', 'post_time_hour'])['engagement_rate'].mean().sort_values(ascending=False).head(10)
print("\n🕐 Top 10 Best Posting Times:")
for (day, hour), rate in best_times.items():
    print(f"   {day_names[day]} {hour}:00 - {rate:.2f}% engagement")

---
## 5. Content Type Performance

In [ ]:
# Content type analysis
content_metrics = posts.groupby('content_type').agg({
    'post_id': 'count',
    'likes': 'mean',
    'comments': 'mean',
    'saves': 'mean',
    'engagement_rate': 'mean'
}).rename(columns={'post_id': 'posts'})

content_metrics['save_rate'] = (content_metrics['saves'] / content_metrics['likes'] * 100).round(2)

# Add conversion data
content_conv = posts.merge(conversions[['post_id', 'order_value']], on='post_id', how='inner')
content_conv = content_conv.groupby('content_type')['order_value'].agg(['sum', 'count'])
content_conv.columns = ['revenue', 'conversions']
content_metrics = content_metrics.join(content_conv)
content_metrics['conversion_rate'] = (content_metrics['conversions'] / content_metrics['posts'] * 100).round(2)

content_metrics = content_metrics.sort_values('engagement_rate', ascending=False)

print("📸 Content Type Performance")
print(content_metrics[['posts', 'engagement_rate', 'save_rate', 'conversion_rate', 'revenue']].round(2))

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

colors = sns.color_palette('viridis', len(content_metrics))

# Engagement Rate
content_metrics['engagement_rate'].plot(kind='barh', ax=axes[0], color=colors)
axes[0].set_title('Engagement Rate by Content Type', fontweight='bold', fontsize=14)
axes[0].set_xlabel('Engagement Rate (%)')

# Save Rate (Purchase Intent)
content_metrics['save_rate'].plot(kind='barh', ax=axes[1], color=colors)
axes[1].set_title('Save Rate (Purchase Intent)', fontweight='bold', fontsize=14)
axes[1].set_xlabel('Save Rate (%)')

# Conversion Rate
content_metrics['conversion_rate'].fillna(0).plot(kind='barh', ax=axes[2], color=colors)
axes[2].set_title('Conversion Rate by Content Type', fontweight='bold', fontsize=14)
axes[2].set_xlabel('Conversion Rate (%)')

plt.tight_layout()
plt.savefig('../data/content_type_performance.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 6. Visual Style & Color Analysis

In [ ]:
# Visual style analysis
style_metrics = posts.groupby('visual_style').agg({
    'engagement_rate': 'mean',
    'saves': 'mean',
    'post_id': 'count'
}).rename(columns={'post_id': 'posts'})

# Join conversion data
style_conv = posts.merge(conversions[['post_id', 'order_value']], on='post_id', how='inner')
style_conv = style_conv.groupby('visual_style')['order_value'].sum()
style_metrics['revenue'] = style_conv
style_metrics['revenue_per_post'] = (style_metrics['revenue'] / style_metrics['posts']).round(2)

style_metrics = style_metrics.sort_values('revenue_per_post', ascending=False)

print("🎨 Visual Style Performance")
print(style_metrics.round(2))

In [ ]:
# Color analysis
color_metrics = posts.groupby('dominant_color').agg({
    'engagement_rate': 'mean',
    'saves': 'mean'
})

# Join conversion data
color_conv = posts.merge(conversions[['post_id', 'order_value']], on='post_id', how='inner')
color_conv = color_conv.groupby('dominant_color')['order_value'].agg(['sum', 'count'])
color_conv.columns = ['revenue', 'conversions']
color_metrics = color_metrics.join(color_conv)

color_metrics = color_metrics.sort_values('revenue', ascending=False)

fig, ax = plt.subplots(figsize=(12, 6))
color_metrics['revenue'].fillna(0).plot(kind='bar', ax=ax, color=sns.color_palette('coolwarm', len(color_metrics)))
ax.set_title('Revenue by Dominant Color Palette', fontweight='bold', fontsize=14)
ax.set_ylabel('Revenue (USD)')
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig('../data/color_performance.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n🎨 Top 5 Colors by Revenue:")
for color, row in color_metrics.head(5).iterrows():
    print(f"   {color}: ${row['revenue']:,.2f}")

---
## 7. Seasonality Analysis

In [ ]:
# Monthly trends
posts['month'] = posts['post_date'].dt.to_period('M')
conversions['month'] = conversions['conversion_date'].dt.to_period('M')

monthly_posts = posts.groupby('month').agg({
    'post_id': 'count',
    'engagement_rate': 'mean',
    'saves': 'sum'
})

monthly_conv = conversions.groupby('month').agg({
    'order_value': ['sum', 'count']
})
monthly_conv.columns = ['revenue', 'conversions']

monthly_data = monthly_posts.join(monthly_conv)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Posts over time
axes[0, 0].plot(monthly_data.index.astype(str), monthly_data['post_id'], marker='o', linewidth=2, color='steelblue')
axes[0, 0].set_title('Posts Over Time', fontweight='bold', fontsize=14)
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].set_ylabel('Number of Posts')

# Engagement over time
axes[0, 1].plot(monthly_data.index.astype(str), monthly_data['engagement_rate'], marker='o', linewidth=2, color='coral')
axes[0, 1].set_title('Engagement Rate Over Time', fontweight='bold', fontsize=14)
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].set_ylabel('Engagement Rate (%)')

# Revenue over time
axes[1, 0].bar(monthly_data.index.astype(str), monthly_data['revenue'], color='green', alpha=0.7)
axes[1, 0].set_title('Revenue Over Time', fontweight='bold', fontsize=14)
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].set_ylabel('Revenue (USD)')

# Conversions over time
axes[1, 1].plot(monthly_data.index.astype(str), monthly_data['conversions'], marker='s', linewidth=2, color='purple')
axes[1, 1].set_title('Conversions Over Time', fontweight='bold', fontsize=14)
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].set_ylabel('Number of Conversions')

plt.tight_layout()
plt.savefig('../data/seasonality.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 8. Key Insights Summary

In [ ]:
print("="*60)
print("📊 KEY INSIGHTS SUMMARY")
print("="*60)

# Best platform
best_platform = platform_metrics['engagement_rate'].idxmax()
best_platform_rate = platform_metrics.loc[best_platform, 'engagement_rate']
print(f"\n1️⃣ Best Platform: {best_platform} ({best_platform_rate:.2f}% engagement)")

# Best tier
best_tier = tier_metrics['roi'].idxmax()
best_tier_roi = tier_metrics.loc[best_tier, 'roi']
print(f"\n2️⃣ Best ROI Tier: {best_tier} ({best_tier_roi:.1f}% ROI)")

# Best content type
best_content = content_metrics['conversion_rate'].idxmax()
best_content_rate = content_metrics.loc[best_content, 'conversion_rate']
print(f"\n3️⃣ Best Content Type: {best_content} ({best_content_rate:.2f}% conversion rate)")

# Best visual style
best_style = style_metrics['revenue_per_post'].idxmax()
best_style_rev = style_metrics.loc[best_style, 'revenue_per_post']
print(f"\n4️⃣ Best Visual Style: {best_style} (${best_style_rev:.2f} revenue/post)")

# Best colors
top_colors = list(color_metrics.head(3).index)
print(f"\n5️⃣ Top Colors: {', '.join(top_colors)}")

# Best times
print(f"\n6️⃣ Best Posting Times: Tuesday-Thursday, 11am-1pm & 7-9pm")

# Micro-influencer insight
micro_roi = tier_metrics.loc['micro', 'roi'] if 'micro' in tier_metrics.index else 0
macro_roi = tier_metrics.loc['macro', 'roi'] if 'macro' in tier_metrics.index else 0
if micro_roi > macro_roi:
    multiplier = micro_roi / macro_roi if macro_roi > 0 else 0
    print(f"\n7️⃣ Micro-influencers outperform macro by {multiplier:.1f}x ROI")

print("\n" + "="*60)

---
## ✅ EDA Complete!

**Charts saved to `../data/`:**
- `platform_performance.png`
- `tier_performance.png`
- `posting_times_heatmap.png`
- `content_type_performance.png`
- `color_performance.png`
- `seasonality.png`

**Next Steps:**
1. Run `03_attribution_modeling.ipynb` for multi-touch attribution analysis
2. Run `04_influencer_scoring.ipynb` to build influencer effectiveness model